In [46]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import torch    
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import yaml
import shutil 
import random
from ultralytics import YOLO

In [2]:
# Iniciar la semilla para la reproducibilidad
# https://pytorch.org/docs/stable/notes/randomness.html
def set_seed(seed): 
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [3]:
# Configuración de la GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


In [ ]:
# Establecer el directorio raíz y los directorios de imágenes y etiquetas
# Cambia la ruta a la ubicación de tu archivo .ipynb
root_dir = os.path.dirname(os.path.abspath("proyecto2.ipynb"))
data_dir = os.path.join(root_dir, "data")
train_imgs_dir = os.path.join(data_dir, "training_images")
train_labels_path = os.path.join(data_dir, "train_solution_bounding_boxes (1).csv")
test_imgs_dir = os.path.join(data_dir, "testing_images")

# Revisar si los directorios y archivos existen
if not os.path.exists(root_dir):
    print(f"El directorio raíz no existe: {data_dir}")
if not os.path.exists(data_dir):
    print(f"El directorio de datos no existe: {data_dir}")
if not os.path.exists(train_imgs_dir):
    print(f"Las imágenes de entrenamiento no existen: {train_imgs_dir}")
if not os.path.exists(train_labels_path):
    print(f"El archivo de etiquetas de entrenamiento no existe: {train_labels_path}")
if not os.path.exists(test_imgs_dir):
    print(f"Las imágenes de prueba no existen: {test_imgs_dir}")

In [5]:
# Cargar las imágenes de entrenamiento
# y mezclar la lista de imágenes
imgs_list = sorted(os.listdir(train_imgs_dir))
random.shuffle(imgs_list)

# Dividir las imágenes en conjuntos de entrenamiento y validación
# 80% para entrenamiento y 20% para validación
split = int(len(imgs_list) * 0.8)
train_imgs = imgs_list[:split]
val_imgs = imgs_list[split:]

print(f"Total de imágenes: {len(imgs_list)}")
print(f"Imágenes de entrenamiento: {len(train_imgs)}")

Total de imágenes: 1001
Imágenes de entrenamiento: 800


In [ ]:
# Establecer los directorios de salida para las imágenes y etiquetas
labels_dir = os.path.join(data_dir, "labels")
images_dir = os.path.join(data_dir, "images")

# Crear los directorios de salida
for sub in ["train", "val"]:
    if not os.path.exists(os.path.join(labels_dir, sub)):
        os.makedirs(os.path.join(labels_dir, sub))
    if not os.path.exists(os.path.join(images_dir, sub)):
        os.makedirs(os.path.join(images_dir, sub))

# Copiar las imágenes de entrenamiento y validación
df = pd.read_csv(train_labels_path)
df.rename(columns={'image': 'img_name'}, inplace=True)
df["class"] = 0

print(f"Total de imágenes de entrenamiento: {len(df)}")

Total de imágenes de entrenamiento: 559


In [7]:
# El tamaño de las imágenes es 676, 380
width, height = 676, 380

# Normalizar las coordenadas de las cajas delimitadoras
df['x_centre'] = (df['xmin'] + df['xmax']) / (2 * width)
df['y_centre'] = (df['ymin'] + df['ymax']) / (2 * height)
df['width'] = (df['xmax'] - df['xmin']) / width
df['height'] = (df['ymax'] - df['ymin']) / height

df_yolo = df[['img_name', 'class', 'x_centre', 'y_centre', 'width', 'height' ]]

In [8]:
df_yolo.head()

,img_name,class,x_centre,y_centre,width,height
0,vid_4_1000.jpg,0,0.450434,0.539817,0.068741,0.095238
1,vid_4_10000.jpg,0,0.100217,0.557191,0.155572,0.129987
2,vid_4_10040.jpg,0,0.444645,0.543678,0.181621,0.157014
3,vid_4_10020.jpg,0,0.833213,0.531451,0.197540,0.155727
4,vid_4_10060.jpg,0,0.110347,0.559122,0.171491,0.136422


In [ ]:
for img_name in imgs_list: 
    # Verificar si la imagen está en el conjunto de entrenamiento o validación
    if img_name in train_imgs:
        subset = "train"
    else:
        subset = "val"

    # Verificar si la imagen ya existe en el directorio de destino
    if os.path.exists(os.path.join(images_dir, subset, img_name)):
        print(f"Ya existe la imagen: {img_name} en {subset}")
        continue

    # Copiar la imagen
    shutil.copy(os.path.join(train_imgs_dir, img_name), os.path.join(images_dir, subset, img_name))

    # Verificar si la etiqueta ya existe en el directorio de destino
    if os.path.exists(os.path.join(labels_dir, subset, img_name.replace(".jpg", ".txt"))):
        print(f"Ya existe la etiqueta: {img_name} en {subset}")
        continue
    
    # Copiar la etiqueta
    if img_name in df_yolo['img_name'].values:
        if img_name in df_yolo["img_name"].values:
            bbox_data = df_yolo[df_yolo["img_name"] == img_name][["class", "x_centre", "y_centre", "width", "height"]].values
            with open(os.path.join(labels_dir, subset, img_name.replace(".jpg", ".txt")), "w") as f:
                for row in bbox_data:
                    f.write(" ".join(map(str, row)) + "\n")

# Verificar si las imágenes y etiquetas se copiaron correctamente
train_imgs_copied = len(os.listdir(os.path.join(images_dir, "train")))
val_imgs_copied = len(os.listdir(os.path.join(images_dir, "val")))

print(f"Imágenes de entrenamiento copiadas: {train_imgs_copied}")
print(f"Imágenes de validación copiadas: {val_imgs_copied}")

Imágenes de entrenamiento copiadas: 800
Imágenes de validación copiadas: 201


In [ ]:
# Realizar predicciones en el conjunto de prueba
yolo_yaml = {
    'path': data_dir, 
    'train': os.path.join(images_dir, "train"),
    'val': os.path.join(images_dir, "val"),
    'nc': 1, 
    'names': ["car"]
}

# Guardar el archivo YAML
if os.path.exists(os.path.join(data_dir, "data.yaml")):
    print(f"Ya existe el archivo data.yaml en {data_dir}")
else:
    print(f"Creando el archivo data.yaml en {data_dir}")
    with open(os.path.join(data_dir, "data.yaml"), "w") as f:
        yaml.dump(yolo_yaml, f, default_flow_style=False)

# Verificar el archivo YAML
with open(os.path.join(data_dir, "data.yaml"), "r") as f:
    data_yaml = yaml.safe_load(f)
    print(yaml.dump(data_yaml, default_flow_style=False))

names:
- car
nc: 1
path: c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data
train: c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\images\train
val: c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\images\val



In [ ]:
model = YOLO('yolov8n.pt') 

# Entrenar el modelo
# Cambia el número de épocas, tamaño de lote y tasa de aprendizaje según sea necesario
model.train(
    data=os.path.join(data_dir, "data.yaml"),
    epochs=10,
    batch=8,
    imgsz=640,
    lr0=0.0005,
    patience=5,
)

# Guardar el modelo entrenado
model.save("yolo_model.pt")

New https://pypi.org/project/ultralytics/8.3.143 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.127  Python-3.13.1 torch-2.6.0+cu126 CPU (Intel Core(TM) i7-10510U 1.80GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False

train: Scanning C:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\labels\train.cache... 276 images, 524 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.20.0 ms, read: 134.858.5 MB/s, size: 92.2 KB)


val: Scanning C:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\labels\val.cache... 79 images, 122 backgrounds, 0 corrupt: 100%|██████████| 201/201 [00:00<?, ?it/s]


Plotting labels to runs\detect\train\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        201        120      0.959      0.963      0.984      0.659



10 epochs completed in 2.281 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.2MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.127  Python-3.13.1 torch-2.6.0+cu126 CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:24<00:00,  1.86s/it]


                   all        201        120      0.959      0.963      0.984       0.66
Speed: 2.0ms preprocess, 98.4ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000021C1AAA7350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [ ]:
# Evaluar el modelo
model = YOLO("runs/yolov8n_custom/weights/best.pt")  
metrics = model.val()

# Mostrar las métricas
print(f"mAP@.5:.95 = {metrics.box.map:.4f}")
print(f"mAP@.50    = {metrics.box.map50:.4f}")
print(f"mAP@.75    = {metrics.box.map75:.4f}")

print(f"Precision   = {metrics.box.mp:.4f}")
print(f"Recall      = {metrics.box.mr:.4f}")


Ultralytics 8.3.127  Python-3.13.1 torch-2.6.0+cu126 CPU (Intel Core(TM) i7-10510U 1.80GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 101.974.1 MB/s, size: 87.2 KB)


val: Scanning C:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\labels\val.cache... 79 images, 122 backgrounds, 0 corrupt: 100%|██████████| 201/201 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:30<00:00,  2.37s/it]


                   all        201        120      0.965      0.942      0.984      0.659
Speed: 2.1ms preprocess, 127.2ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs\detect\val2
mAP@.5:.95 = 0.6587
mAP@.50    = 0.9843
mAP@.75    = 0.7826
Precision   = 0.9647
Recall      = 0.9417


In [ ]:
# Crear un directorio para guardar las predicciones
if not os.path.exists(os.path.join(root_dir, "runs", "yolov8n_custom", "predictions")):
    print(f"Creando el directorio para guardar las predicciones: {os.path.join(root_dir, 'runs', 'yolov8n_custom', 'predictions')}")
    %mkdir {os.path.join(root_dir, "runs", "yolov8n_custom", "predictions")}

# Realizar predicciones en el conjunto de prueba
# Cambia el umbral de confianza y IOU según sea necesario
results = model.predict(
    source=test_imgs_dir,
    conf=0.5,
    iou=0.75)

# Mostrar las predicciones
test_img_list = []
for result in results:
    if len(result.boxes.xyxy):
        name = os.path.basename(result.path).split('.')[0]
        boxes = result.boxes.xyxy.cpu().numpy()
        scores = result.boxes.conf.cpu().numpy()
        test_img_list.append(name)

        with open(os.path.join(root_dir, "runs/yolov8n_custom/predictions", f"{name}.txt"), "w") as f:
            for score, box in zip(scores, boxes):
                # Escribir la puntuación y las coordenadas de la caja delimitadora
                f.write(f"{score:.4f} {' '.join(map(str, box))}\n")  

Ya existe el subdirectorio o el archivo c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\runs\yolov8n_custom\predictions.


image 1/175 c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\testing_images\vid_5_25100.jpg: 384x640 (no detections), 193.1ms
image 2/175 c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\testing_images\vid_5_25120.jpg: 384x640 (no detections), 275.2ms
image 3/175 c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\testing_images\vid_5_25140.jpg: 384x640 (no detections), 142.4ms
image 4/175 c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\testing_images\vid_5_25160.jpg: 384x640 (no detections), 117.9ms
image 5/175 c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\testing_images\vid_5_25180.jpg: 384x640 (no detections), 169.2ms
image 6/175 c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\testing_images\vid_5_25200.jpg: 384x640 (no detections), 127.5ms
image 7/175 c:\Users\ivand\Documents\GitHub\proyecto-final-redes-neuronales\data\testing_images\vid_5_25220.jpg: 384x640 (no

In [ ]:
# Mostrar las imágenes con las predicciones
def show_bbox(img, boxes, scores, axis, color=(0,255,0)):
    img = img.copy()
    # Verificar si hay cajas delimitadoras
    if len(boxes) == 0:
        return
    # Dibujar las cajas delimitadoras
    for i, box in enumerate(boxes.astype(int)):
        if scores[i] < 0.5:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
        y = box[1] - 10 if box[1] - 10 > 10 else box[1] + 10
        cv2.putText(img, f"{scores[i]:.2f}", (box[0], y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    axis.imshow(img)
    axis.axis("off")


In [ ]:
# Número de imágenes a mostrar
# Cambia el número de imágenes a mostrar según sea necesario
fig, axes = plt.subplots(10, 2, figsize=(16, 20))
axes = axes.flatten()

# Seleccionar aleatoriamente 20 imágenes de prueba
sample_imgs = np.random.choice(test_img_list, 20, replace=False)

# Mostrar imágenes de prueba con predicciones
for i, img_name in enumerate(sample_imgs):
    img_path = os.path.join(test_imgs_dir, img_name + ".jpg")
    label_path = os.path.join(root_dir, "runs/yolov8n_custom/predictions", f"{img_name}.txt")

    # Leer la imagen
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Verificar si la imagen existe
    if os.path.exists(label_path):  
        # Leer las predicciones
        label = pd.read_csv(label_path, sep=" ", header=None).values
        show_bbox(img, label[:, 1:], label[:, 0], axes[i])
    else:
        # Mostrar la imagen sin predicciones
        axes[i].imshow(img)
        axes[i].axis("off")
        axes[i].set_title("No predictions")

plt.tight_layout()
plt.show()
plt.savefig(os.path.join(root_dir, "runs/yolov8n_custom", "predictions", "predictions.png"), dpi=300)



<Figure size 640x480 with 0 Axes>

<Figure size 1600x2000 with 20 Axes>